# Single Drug Prediction Workflow

This is the interactive version of the single drug prediction workflow using PyCOMPSs.

It uses the Building Blocks from jupyter, whilst PyCOMPSs is used to parallelize the workflow.

In [ ]:
import os
import shutil
from pathlib import Path

current_path = os.getcwd()
if not "PERMEDCOE_IMAGES" in os.environ:
    os.environ["PERMEDCOE_IMAGES"] = current_path + "/../../../../BuildingBlocks/Resources/images/"
os.environ["COMPUTING_UNITS"] = "1"

First, start the PyCOMPSs runtime and import the used PyCOMPSs synchronization function (``compss_wait_on_directory``)

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
ipycompss.start(graph=True, trace=False)

# PyCOMPSs imports
from pycompss.api.api import compss_wait_on_file

Second, import the required Building Blocks

In [ ]:
# To set the default PyCOMPSs TMPDIR
from permedcoe import TMPDIR
# Import building block tasks
from Carnival_gex_preprocess_BB import preprocess as carnival_gex_preprocess
from progeny_BB import progeny
from omnipath_BB import omnipath
from tf_enrichment_BB import tf_enrichment
from CarnivalPy_BB import carnivalpy
from Carnival_feature_merger_BB import feature_merger as carnival_feature_merger
from ml_jax_drug_prediction_BB import ml as ml_jax_drug_prediction
# TODO: carnival_BB not used
# TODO: cellnopt_BB not used
# TODO: export_solver_hdf5 not used

Third, set the input parameters and check results folder

In [ ]:
print("---------------------------------------")
print("|   SINGLE DRUG PREDICTION Workflow   |")
print("---------------------------------------")

results = current_path + "/results/"
results_csvs = current_path + "/results_csvs/"
Path(current_path + "/dummy.x").touch()

cell_list = current_path + "/../../../Resources/data/cell_list_example.txt"
gene_expression = current_path + "/../../../Resources/data/Cell_line_RMA_proc_basalExp.txt"
gex = ""
gex_n = ""
progeny_file = ""
network = ""
genelist = results_csvs + "/genelist.csv"
jax_input = "dummy.x"
results_folder = results
results_csvs_folder = results_csvs

In [ ]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder, exist_ok=True)
else:
    # Do not continue if results folder exists
    raise Exception("ERROR: Results folder exists: %s" % results_folder)
if not os.path.exists(results_csvs_folder):
    os.makedirs(results_csvs_folder, exist_ok=True)
else:
    # Do not continue if results csvs folder exists
    raise Exception("ERROR: Results CSVs folder exists: %s" % results_csvs_folder)

Initial STEP: Read the list of cells to process

In [ ]:
with open(cell_list, 'r') as cell_list_fd:
    cells_raw = cell_list_fd.readlines()
    cells = [cell.strip() for cell in cells_raw]

1st STEP: Remove the DATA. prefix from the columns

In [ ]:
if gex:
    gex_csv = gex
else:
    gex_csv = os.path.join(results_folder, "gex.csv")
if not os.path.exists(gex_csv):
    if gene_expression:
        print("INFO: gex does not exist. Creating in: %s using %s" % (gex_csv, gene_expression))
        carnival_gex_preprocess(tmpdir=TMPDIR,
                                input_file=gene_expression,
                                output_file=gex_csv,
                                col_genes="GENE_SYMBOLS",
                                scale="FALSE",
                                exclude_cols="GENE_title",
                                tsv="TRUE",
                                remove="DATA.",
                                verbose="TRUE"
        )
    else:
        raise Exception("ERROR: Please provide --gene_expression if --gex does not exist or is not defined")
else:
    print("INFO: Using existing gex: %s" % gex_csv)

2nd STEP: Do the same but this time scale also genes across cell lines

In [ ]:
if gex_n:
    gex_n_csv = gex_n
else:
    gex_n_csv = os.path.join(results_folder, "gex_n.csv")
if not os.path.exists(gex_n_csv):
    print("INFO: gex_n does not exist. Creating in: %s using %s" % (gex_n_csv, gex_csv))
    carnival_gex_preprocess(tmpdir=TMPDIR,
                            input_file=gex_csv,
                            output_file=gex_n_csv,
                            col_genes="GENE_SYMBOLS",
                            scale="TRUE",
                            exclude_cols="GENE_title",
                            tsv="FALSE",
                            remove="DATA.",
                            verbose="TRUE"
    )
else:
    print("INFO: Using existing gex_n: %s" % gex_n_csv)

3rd STEP: Use the gene expression data to run Progeny and estimate pathway activities

In [ ]:
if progeny_file:
    progeny_csv = progeny_file
else:
    progeny_csv = os.path.join(results_folder, "progeny.csv")
if not os.path.exists(progeny_csv):
    print("INFO: progeny does not exist. Creating in: %s using %s" % (progeny_csv, gex_csv))
    progeny(tmpdir=TMPDIR,
            input_file=gex_csv,
            output_file=progeny_csv,
            organism="Human",
            ntop="100",
            col_genes="GENE_SYMBOLS",
            scale="TRUE",
            exclude_cols="GENE_title",
            tsv="FALSE",
            perms="1",
            zscore="FALSE",
            verbose="TRUE"
    )
else:
    print("INFO: Using existing progeny: %s" % progeny_csv)

4th STEP: Get SIF from omnipath

In [ ]:
if network:
    network_csv = network
else:
    network_csv = os.path.join(results_folder, "network.csv")
if not os.path.exists(network_csv):
    print("INFO: network does not exist. Creating in: %s" % network_csv)
    omnipath(tmpdir=TMPDIR,
             debug=False,
             output_file=network_csv
    )
else:
    print("INFO: Using existing network: %s" % network_csv)

In [ ]:
for cell in cells:
    print("Processing %s" % cell)
    cell_result_folder = os.path.join(results_folder, cell)
    os.makedirs(cell_result_folder, exist_ok=True)

    # 5th STEP: Run CARNIVAL on the sample and extract the features
    cell_measurement_csv = os.path.join(cell_result_folder, "measurements.csv")
    tf_enrichment(tmpdir=TMPDIR,
                  input_file=gex_n_csv,
                  output_file=cell_measurement_csv,
                  tsv="FALSE",
                  weight_col=cell,
                  id_col="GENE_SYMBOLS",
                  minsize=10,
                  source="tf",
                  confidence="A,B,C",
                  verbose="TRUE",
                  pval_threshold=0.1,
                  export_carnival="TRUE"
    )


In [ ]:
compss_wait_on_file(network_csv)
for cell in cells:
    cell_result_folder = os.path.join(results_folder, cell)
    cell_measurement_csv = os.path.join(cell_result_folder, "measurements.csv")
    shutil.copyfile(network_csv, cell_result_folder + "/network.csv")
    compss_wait_on_file(cell_measurement_csv)

In [ ]:
for cell in cells:
    # 6th STEP: Run CarnivalPy on the sample using cbc
    cell_result_folder = os.path.join(results_folder, cell)
    carnival_csv = os.path.join(cell_result_folder, "carnival.csv")
    carnivalpy(tmpdir=TMPDIR,
               path=cell_result_folder,
               penalty=0,
               solver="cbc",
               tol=0.1,
               maxtime=500,
               export=carnival_csv,
    )

In [ ]:
compss_wait_on_file(progeny_csv)
for cell in cells:
    cell_result_folder = os.path.join(results_folder, cell)
    carnival_csv = os.path.join(cell_result_folder, "carnival.csv")
    compss_wait_on_file(carnival_csv)

7th STEP: Merge the features into a single CSV, focus only on some specific genes

In [ ]:
cell_features_csv = os.path.join(results_csvs_folder, "cell_features.csv")
carnival_feature_merger(tmpdir=TMPDIR,
                        input_dir=results_folder,
                        output_file=cell_features_csv,
                        feature_file=genelist,
                        merge_csv_file=progeny_csv,
                        merge_csv_index="sample",
                        merge_csv_prefix="F_"
)

8th STEP: Train a model to predict IC50 values for unknown cells (using the progeny+carnival features) and known drugs

In [ ]:
model_npz = os.path.join(results_csvs_folder, "model.npz")
ml_jax_drug_prediction(tmpdir=TMPDIR,
                       input_file=jax_input,
                       output_file=model_npz,
                       drug_features=".none",
                       cell_features=cell_features_csv,
                       epochs="200",
                       adam_lr="0.1",
                       reg="0.001",
                       latent_size="10",
                       test_drugs="0.1",
                       test_cells="0.1"
)
compss_wait_on_file(model_npz)

Finally, we can stop the PyCOMPSs runtime.

In [ ]:
ipycompss.stop(sync=False)